In [92]:
import json
import requests
import datetime
import pandas as pd
import urllib.request
import codecs
import numpy as np
from six.moves import urllib

In [93]:
#eligibility criteria
volume_cutoff = 0.1
percent_float_cutoff = 0.1

#month of rebalance
old_month = 'april'
new_month = 'may'

In [94]:
#change call to new filepath
asset_db = pd.read_excel('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/utils/rebalance_tools/csvs\\Bletchley Rebalance '+new_month+' 2019.xlsx',\
                         sheetname='assets',index_col='Ticker')
#change call to new filepath
coinmarketcap_db = pd.read_excel('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/utils/rebalance_tools/csvs\\Bletchley Rebalance '+new_month+' 2019.xlsx',\
                         sheetname='coinmarketcap',index_col='Ticker')

In [95]:
#grab all assets from Messari
#deprecated as they have too many!
#url = "https://data.messari.io/api/v1/assets"
#response = (urllib.request.urlopen(url).read())
#response = response.decode('utf-8')
#d = json.loads(response)
#df = pd.DataFrame(d['data'])
#df = df.set_index((df['symbol']))

In [96]:
df = asset_db
#ticker mapping issues between Bletchley DB and OnChainFX
df.loc['VEN'] = df.loc['VET']
df.index = map(lambda x: x.lower(), df.index)

In [97]:
#get Messari data given a ticker
#don't actually use the category or sector data
def get_messari_data(ticker):
    url_messari = "https://data.messari.io/api/v1/assets/{}/metrics".format(ticker)
    response_messari = (urllib.request.urlopen(url_messari).read())
    response_messari = response_messari.decode('utf-8')
    d_messari = json.loads(response_messari)
    df_all = pd.Series()
    df_all['Y2050 Supply'] = (d_messari['data']['supply']['y_2050'])
    df_all['Available Supply'] = (d_messari['data']['supply']['circulating'])
    df_all['Supply % Issued'] = df_all['Available Supply'] / (df_all['Y2050 Supply'])
    df_all['Price'] = (d_messari['data']['market_data']['price_usd'])
    df_all['Marketcap'] = df_all['Price'] * df_all['Y2050 Supply']
    df_all['Current Marketcap'] = df_all['Price'] * df_all['Available Supply']
    df_all['Category'] = (d_messari['data']['misc_data']['categories'][0])
    df_all['Sectors'] = (d_messari['data']['misc_data']['sectors'][0])
    try:
        df_all['Category2'] = (d_messari['data']['misc_data']['categories'][1])
    except:
        pass
    try:
        df_all['Sectors2'] = (d_messari['data']['misc_data']['sectors'][1])
    except:
        pass
    return df_all

In [98]:
#master function to get all Messari data and put in dictionary
metrics_log = dict.fromkeys(df.index)
for t in df.index:
    try:
        metrics = get_messari_data(t)
        metrics_log[t] = metrics
        df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
        df.loc[t,('Available Supply')] = metrics['Available Supply']
        df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
        df.loc[t,('Price')] = metrics['Price']
        df.loc[t,('Marketcap')] = metrics['Marketcap']
        df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
        df.loc[t,('Category')] = metrics['Category']
        df.loc[t,('Sectors')] = metrics['Sectors']
        try:
            df.loc[t,('Category2')] = metrics['Category2']
        except:
            pass
        try:
            df.loc[t,('Sectors2')] = metrics['Sectors2']
        except:
            pass
    except:
        pass

C:\Users\stoin\Anaconda2\envs\py35\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in longlong_scalars
  # This is added back by InteractiveShellApp.init_path()


In [134]:
t = 'IOTA'
metrics = get_messari_data(t)
t = 'IOT'
metrics_log[t] = metrics
df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
df.loc[t,('Available Supply')] = metrics['Available Supply']
df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
df.loc[t,('Price')] = metrics['Price']
df.loc[t,('Marketcap')] = metrics['Marketcap']
df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
df.loc[t,('Category')] = metrics['Category']
df.loc[t,('Sectors')] = metrics['Sectors']
try:
    df.loc[t,('Category2')] = metrics['Category2']
except:
    pass
try:
    df.loc[t,('Sectors2')] = metrics['Sectors2']
except:
    pass

In [135]:
t = 'VET'
metrics = get_messari_data(t)
metrics_log[t] = metrics
df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
df.loc[t,('Available Supply')] = metrics['Available Supply']
df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
df.loc[t,('Price')] = metrics['Price']
df.loc[t,('Marketcap')] = metrics['Marketcap']
df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
df.loc[t,('Category')] = metrics['Category']
df.loc[t,('Sectors')] = metrics['Sectors']
try:
    df.loc[t,('Category2')] = metrics['Category2']
except:
    pass
try:
    df.loc[t,('Sectors2')] = metrics['Sectors2']
except:
   pass

In [136]:
df.loc['IOT']

,Coin,Class,Sector,Quality Exchange?,ERC20,ICO ok?,Eligible,Y2050 Supply,Available Supply,Supply % Issued,Price,Marketcap,Current Marketcap,Category,Sectors,Sectors2,Category2
IOT,IOTA,1.0,NaN,1.0,0.0,1.0,1.0,2.779530e+09,2.779530e+09,1.0,0.292924,8.141914e+08,8.141914e+08,Infrastructure,Internet of Things,NaN,NaN
IOT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.779530e+09,2.779530e+09,1.0,0.292924,8.141914e+08,8.141914e+08,Infrastructure,Internet of Things,NaN,NaN


In [137]:
#change Messari data to uppercase to match other sources
df.index = map(lambda x: x.upper(), df.index)

In [138]:
#df.loc['IOT'] = df.loc['IOTA']
#df.loc['VET'] = df.loc['VEN']

In [139]:
#read in other data sources
onchainfx_db = df

In [140]:
#remove duplicates
print (len(coinmarketcap_db))
print (len(onchainfx_db))
print (len(asset_db))
coinmarketcap_db = coinmarketcap_db[~coinmarketcap_db.index.duplicated(keep='first')]
onchainfx_db = onchainfx_db[~onchainfx_db.index.duplicated(keep='first')]
asset_db = asset_db[~asset_db.index.duplicated(keep='first')]
print (len(coinmarketcap_db))
print (len(onchainfx_db))
print (len(asset_db))

691
147
145
691
145
145


In [141]:
#fix any ticker mapping issues
onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})

In [143]:
onchainfx_db.loc['IOT']

Coin                               IOTA
Class                                 1
Sector                              NaN
Quality Exchange?                     1
ERC20                                 0
ICO ok?                               1
Eligible                              1
Y2050 Supply                2.77953e+09
Available Supply            2.77953e+09
Supply % Issued                       1
Price                          0.292924
Marketcap                   8.14191e+08
Current Marketcap           8.14191e+08
Category                 Infrastructure
Sectors              Internet of Things
Sectors2                            NaN
Category2                           NaN
Name: IOT, dtype: object

In [144]:
#onchainfx_db = pd.read_excel('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/utils/rebalance_tools/csvs\\ocfx_'+new_month+'_2019.xlsx',\
#index_col='Ticker')

In [145]:
#read in previous month Bletchley Data
list_indexes = ['10' , '20' , '40' , 'Total' , 'ETH' , 'Currency' , 'Platform' , 'Application']
indexes_old = dict.fromkeys(list_indexes)
indexes_new_tickers = dict.fromkeys(list_indexes)
indexes_new_file = dict.fromkeys(list_indexes)
for k in indexes_old:
    if (k == 'Currency' or k == 'Platform' or k == 'Application'):
        indexes_old[k] = pd.DataFrame()
        indexes_new_tickers[k] = pd.DataFrame()
        indexes_new_file[k] = pd.DataFrame(columns=['Float'])
        indexes_old[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+old_month+'_rebal.csv' , index_col=[0] , names=[k,'Adj Float'])
    else:
        indexes_old[k] = pd.DataFrame()
        indexes_new_tickers[k] = pd.DataFrame()
        indexes_new_file[k] = pd.DataFrame(columns=['Float'])
        indexes_old[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+old_month+'_rebal.csv' , index_col=[0] , names=[k,'Float' , 'Adj Float'])

In [146]:
#main function to create table of all assets
def create_universe(onchainfx,coinmarketcap,assets):
    eligibility = pd.DataFrame(columns = ['Marketcap'])
    eligibility['Marketcap'] = onchainfx['Marketcap']
    eligibility['Current Marketcap'] = onchainfx['Current Marketcap']
    eligibility['Float Percent'] = onchainfx['Supply % Issued']
    eligibility['Float'] = onchainfx['Available Supply']
    eligibility['Price'] = onchainfx['Price']
    eligibility['Total Supply'] = onchainfx['Y2050 Supply']
    eligibility['Float Pass'] = np.where(eligibility['Float Percent']>=0.1, 1, 0)
    for t in eligibility.index:
        if t in coinmarketcap.index:
            eligibility.loc[t,('Volume')] = coinmarketcap.loc[t,('Volume (30d)')]
        else:
            print (t,' no volume data!')
            eligibility.loc[t,('Volume')] = 1
    eligibility['Volume Ratio'] = eligibility['Volume'] / eligibility['Current Marketcap']
    eligibility['Volume Pass'] = np.where(eligibility['Volume Ratio']>=volume_cutoff, 1, 0)
    for t in eligibility.index:
        if t in assets.index:
            eligibility.loc[t,('Other Pass')] = assets.loc[t,('Eligible')]
            eligibility.loc[t,('Class')] = assets.loc[t,('Class')]
            eligibility.loc[t,('ERC20')] = assets.loc[t,('ERC20')]
        else:
            print (t,' no other data!')
            eligibility.loc[t,('Other Pass')] = 0
    eligibility['Score'] = eligibility['Volume Pass'] + eligibility['Other Pass'] + eligibility['Float Pass']
    eligibility['Eligible'] = np.where(eligibility['Score']>=2.99999, 1, 0)
    #check if duplicates
    count = len(eligibility.index)
    count1 = len(eligibility.index.unique())
    if count != count1:
        print ('duplicate tickers')
    return eligibility

In [147]:
#rank assets by 2050 marketcap for selection list
def rank_and_sort(df):
    df['Rank'] = df['Marketcap'].rank(ascending=False).astype(int)
    df_sorted = df.sort_values('Rank')
    return df_sorted

In [148]:
#get list of eligible assets and rank them
#really noisy error logging right now due to Messari having 4000 assets
eligibility_db = create_universe(onchainfx_db,coinmarketcap_db,asset_db)
eligible_db = eligibility_db[eligibility_db.Eligible != 0]
ineligible_db = eligibility_db[eligibility_db.Eligible == 0]

ANT  no volume data!
CENNZ  no volume data!
ECN  no volume data!
EDGE  no volume data!
GBYTE  no volume data!
HC  no volume data!
HC  no volume data!
LBC  no volume data!
MCAP  no volume data!
MLN  no volume data!
NMR  no volume data!
PLR  no volume data!
POT  no volume data!
SPANK  no volume data!
VEN  no volume data!
duplicate tickers


In [149]:
#make sure I've correctly excluded all of these
double_check = ineligible_db[ineligible_db['Volume Pass'] == 1]
double_check
ineligible_db

,Marketcap,Current Marketcap,Float Percent,Float,Price,Total Supply,Float Pass,Volume,Volume Ratio,Volume Pass,Other Pass,Class,ERC20,Score,Eligible
ANT,2.446776e+07,1.890427e+07,0.772619,3.060309e+07,0.617724,3.960952e+07,1,1.0,5.289811e-08,0,1.0,3.0,1.0,2.0,0
BCN,1.686226e+08,1.682541e+08,0.997814,1.840668e+11,0.000914,1.844700e+11,1,3831870.0,2.277431e-02,0,0.0,1.0,0.0,1.0,0
BLOCK,5.517788e+07,1.453887e+07,0.263491,5.742454e+06,2.531821,2.179375e+07,1,400000.0,2.751246e-02,0,1.0,2.0,0.0,2.0,0
BLZ,NaN,NaN,NaN,NaN,NaN,NaN,0,15502416.0,NaN,0,1.0,2.0,0.0,1.0,0
BTCP,2.807500e+07,2.765982e+07,0.985212,2.268143e+07,1.219492,2.302189e+07,1,1408103.0,5.090789e-02,0,0.0,1.0,0.0,1.0,0
BTM,2.121724e+08,1.012870e+08,0.477381,1.002499e+09,0.101034,2.100000e+09,1,84940479.0,8.386118e-01,1,0.0,2.0,0.0,2.0,0
CENNZ,3.831743e+07,2.566096e+07,0.669694,8.036329e+08,0.031931,1.200000e+09,1,1.0,3.896971e-08,0,0.0,3.0,0.0,1.0,0
CND,NaN,NaN,NaN,NaN,NaN,NaN,0,6798910.0,NaN,0,1.0,3.0,0.0,1.0,0
CNX,NaN,NaN,NaN,NaN,NaN,NaN,0,155457775.0,NaN,0,0.0,3.0,0.0,0.0,0
CTXC,4.620983e+07,2.308892e+07,0.499654,1.497925e+08,0.154139,2.997925e+08,1,68819301.0,2.980620e+00,1,0.0,2.0,0.0,2.0,0


In [150]:
#create eligible universe - not sure on why there is a warning
universe = rank_and_sort(eligible_db)

C:\Users\stoin\Anaconda2\envs\py35\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [151]:
universe = universe.rename({'HSR' : 'HC'})
universe

,Marketcap,Current Marketcap,Float Percent,Float,Price,Total Supply,Float Pass,Volume,Volume Ratio,Volume Pass,Other Pass,Class,ERC20,Score,Eligible,Rank
BTC,1.113967e+11,9.383249e+10,0.842327,1.767498e+07,5308.776203,2.098350e+07,1,1.924724e+11,2.051234,1,1.0,1.0,0.0,3.0,1,1
XRP,3.064209e+10,1.287120e+10,0.420050,4.200497e+10,0.306421,1.000000e+11,1,1.638648e+10,1.273112,1,1.0,2.0,0.0,3.0,1,2
ETH,2.005298e+10,1.711557e+10,0.853518,1.058802e+08,161.650301,1.240516e+08,1,9.705392e+10,5.670503,1,1.0,2.0,0.0,3.0,1,3
XLM,1.435470e+10,1.922414e+09,0.133922,1.910280e+10,0.100635,1.426409e+11,1,4.197937e+09,2.183680,1,1.0,1.0,0.0,3.0,1,4
EOS,8.260921e+09,4.506589e+09,0.545531,9.438770e+08,4.774551,1.730198e+09,1,2.927942e+10,6.497026,1,1.0,2.0,0.0,3.0,1,5
LTC,6.205190e+09,4.554697e+09,0.734014,6.157166e+07,73.973926,8.388348e+07,1,3.052390e+10,6.701630,1,1.0,1.0,0.0,3.0,1,6
BCH,5.742581e+09,4.859719e+09,0.846260,1.775745e+07,273.672128,2.098343e+07,1,8.648583e+09,1.779647,1,1.0,1.0,0.0,3.0,1,7
ADA,2.932639e+09,1.808833e+09,0.616793,2.592707e+10,0.069766,4.203526e+10,1,6.126591e+08,0.338704,1,1.0,2.0,0.0,3.0,1,8
TRX,2.643529e+09,1.570441e+09,0.594070,6.668207e+10,0.023551,1.122462e+11,1,6.263859e+09,3.988599,1,1.0,3.0,1.0,3.0,1,9
XTZ,2.303177e+09,9.631579e+08,0.418187,7.927195e+08,1.215005,1.895612e+09,1,2.900000e+08,0.301093,1,1.0,2.0,0.0,3.0,1,10


In [152]:
#create ten index membership
ten_auto = universe[universe['Rank'] <= 8]
indexes_new_tickers['10'] = pd.DataFrame()
indexes_new_tickers['10'] = ten_auto
ten_band = universe[universe['Rank'] <= 12]
ten_band = ten_band[ten_band['Rank'] > 8]
count_ten_members = len(ten_auto)
for t in ten_band.index:
    if t in indexes_old['10'].index:
        if count_ten_members < 10:
            indexes_new_tickers['10'] = indexes_new_tickers['10'].append(ten_band.loc[t])
            count_ten_members += 1
            print ('Retained' , t)
        else:
            print ('10 full! No room for' , t)
for t in ten_band.index:
    if t not in indexes_old['10'].index:
        if count_ten_members < 10:
            print ('New add' , t)
            indexes_new_tickers['10'] = indexes_new_tickers['10'].append(ten_band.loc[t])
            count_ten_members += 1
        else:
            print ('10 full! Do not add ' , t)

Retained TRX
New add XTZ
10 full! Do not add  BNB
10 full! Do not add  DASH


In [153]:
#create twenty index membership
twenty_auto = universe[universe['Rank'] <= 26]
for t in twenty_auto.index:
    if t in indexes_new_tickers['10'].index:
        twenty_auto = twenty_auto.drop(t)
#remove all rows in indexes_new_tickers['10']
indexes_new_tickers['20'] = pd.DataFrame()
indexes_new_tickers['20'] = twenty_auto
twenty_band = universe[universe['Rank'] <= 34]
twenty_band = twenty_band[twenty_band['Rank'] > 26]
count_twenty_members = len(twenty_auto)
for t in twenty_band.index:
    if t in indexes_old['20'].index:
        if count_twenty_members < 20:
            indexes_new_tickers['20'] = indexes_new_tickers['20'].append(twenty_band.loc[t])
            count_twenty_members += 1
            print ('Retained' , t)
        else:
            print ('20 full! No room for' , t)
for t in twenty_band.index:
    if t not in indexes_old['20'].index:
        if count_twenty_members < 20:
            print ('New add' , t)
            indexes_new_tickers['20'] = indexes_new_tickers['20'].append(twenty_band.loc[t])
            count_twenty_members += 1
        else:
            print ('20 full! Do not add ' , t)

Retained LINK
Retained MANA
Retained LSK
Retained ZIL
20 full! No room for ZRX
20 full! Do not add  BTG
20 full! Do not add  QKC
20 full! Do not add  ICX


In [154]:
#create forty index membership
forty_auto = universe[universe['Rank'] <= 60]
for t in forty_auto.index:
    if t in indexes_new_tickers['10'].index:
        forty_auto = forty_auto.drop(t)
    if t in indexes_new_tickers['20'].index:
        forty_auto = forty_auto.drop(t)
indexes_new_tickers['40'] = pd.DataFrame()
indexes_new_tickers['40'] = forty_auto
forty_band = universe[universe['Rank'] <= 80]
forty_band = forty_band[forty_band['Rank'] > 60]
count_forty_members = len(forty_auto)
for t in forty_band.index:
    if t in indexes_old['40'].index:
        if count_forty_members < 40:
            indexes_new_tickers['40'] = indexes_new_tickers['40'].append(forty_band.loc[t])
            count_forty_members += 1
            print ('Retained' , t)
        else:
            print ('40 full! No room for' , t)
for t in forty_band.index:
    if t not in indexes_old['40'].index:
        if count_forty_members < 40:
            indexes_new_tickers['40'] = indexes_new_tickers['40'].append(forty_band.loc[t])
            count_forty_members += 1
            print ('New add' , t)
        else:
            print ('40 full! Do not add ' , t)

Retained POWR
Retained NAS
Retained STRAT
Retained STORJ
Retained THETA
Retained PIVX
Retained POLY
Retained LRC
Retained GNT
Retained PPT
40 full! Do not add  MONA
40 full! Do not add  GXS
40 full! Do not add  IOTX
40 full! Do not add  WAN
40 full! Do not add  CVC
40 full! Do not add  ARDR
40 full! Do not add  ENG
40 full! Do not add  LOOM
40 full! Do not add  FUN
40 full! Do not add  ARK


In [155]:
#construct total index from 10 + 20 + 40
indexes_new_tickers['Total'] = indexes_new_tickers['10'].append(indexes_new_tickers['20']).append(indexes_new_tickers['40'])

In [156]:
#construct various sector indexes from total index
indexes_new_tickers['ETH'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['ERC20'] == 1]
indexes_new_tickers['Currency'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 1]
indexes_new_tickers['Platform'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 2]
indexes_new_tickers['Application'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 3]

In [157]:
#print out membership changes for checking and blog
for k in indexes_old:
    print (k , 'adds' , indexes_new_tickers[k].index.difference(indexes_old[k].index).values)
    print (k , 'deletes' , indexes_old[k].index.difference(indexes_new_tickers[k].index).values)
    print (k , 'count members:' , len(indexes_new_tickers[k]))
    if indexes_new_tickers[k].index.duplicated(keep='first').any() == True:
        print ('Duplicates if True' , indexes_new_tickers[k].index.duplicated(keep='first'))
    print ("")
sectors = len(indexes_new_tickers['Currency'])+len(indexes_new_tickers['Platform'])+len(indexes_new_tickers['Application'])
print ('Sector count members:' , sectors)

ETH adds ['WAX']
ETH deletes []
ETH count members: 24

Application adds ['WAX']
Application deletes ['RDD']
Application count members: 25

Currency adds []
Currency deletes []
Currency count members: 18

20 adds ['BAT' 'BSV']
20 deletes ['XTZ' 'ZRX']
20 count members: 20

10 adds ['XTZ']
10 deletes ['BSV']
10 count members: 10

40 adds ['QKC' 'WAX' 'ZRX']
40 deletes ['BAT' 'HOT' 'RDD']
40 count members: 40

Total adds ['QKC' 'WAX']
Total deletes ['HOT' 'RDD']
Total count members: 70

Platform adds ['QKC']
Platform deletes ['HOT']
Platform count members: 27

Sector count members: 70


In [158]:
#write off new index memberships for use in rebalance
for k in indexes_new_tickers:
    indexes_new_tickers[k].to_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_pre_'+new_month+'.csv',columns=['Float'])